# 1 神经网络

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [2]:
data = loadmat('ex4data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
X = data['X']
y = data['y']

X.shape,y.shape

((5000, 400), (5000, 1))

我们也需要对我们的y标签进行一次one-hot 编码。 one-hot 编码将类标签n（k类）转换为长度为k的向量，其中索引n为“hot”（1），而其余为0。 Scikitlearn有一个内置的实用程序，我们可以使用这个。

In [4]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse = False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [5]:
y[0],y_onehot[0,:]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

# sigmoid 函数
g 代表一个常用的逻辑函数（logistic function）为S形函数（Sigmoid function），公式为： \\[g\left( z \right)=\frac{1}{1+{{e}^{-z}}}\\] 
合起来，我们得到逻辑回归模型的假设函数： 
	\\[{{h}_{\theta }}\left( x \right)=\frac{1}{1+{{e}^{-{{\theta }^{T}}X}}}\\] 

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 前向传播函数
> (400 + 1) -> (25 + 1) -> (10)

<img style="float: left;" src="nn_model.png">

In [7]:
def forward_propagate(X,theta1,theta2):
    
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values = np.ones(m),axis = 1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values = np.ones(m),axis = 1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h
 

# 代价函数
Tip：重新定义矩阵维度可以使用reshape函数
<img style="float: left;" src="nn_regcost.png">


In [8]:
def cost(params,input_size,hidden_size,num_labels,X,y,lamda):
    
    m = X.shape[0]
    
    X = np.matrix(X)
    y = np.matrix(y)
    
    #按照输入层和隐藏层的维度定义theta的维度
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size,(input_size +1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels,(hidden_size +1))))

    a1, z2, a2, z3, h = forward_propagate(X,theta1,theta2)
    
    J = 0
    
    for i in range(m):
        first_term = np.multiply(-y[i,:],np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]),np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
        
    J = J / m
    
    J += (float(lamda) / (2 * m)) * (np.sum(np.power(theta1[:,1:],2)) + np.sum(np.power(theta2[:,1:],2))) 
    
    return J

    

In [9]:
input_size = 400
hidden_size = 25
num_labels = 10
lamda = 1

params = (np.random.random(size = hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5)* 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size,(input_size +1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels,(hidden_size +1))))



theta1.shape,theta2.shape

((25, 401), (10, 26))

In [10]:
a1, z2, a2, z3, h = forward_propagate(X,theta1,theta2)

a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [11]:
cost(params,input_size,hidden_size,num_labels,X,y_onehot,lamda)


7.652254720789496

# 反向传播法
接下来是反向传播算法。 反向传播参数更新计算将减少训练数据上的网络误差。 我们需要的第一件事是计算我们之前创建的Sigmoid函数的梯度的函数。

In [12]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [13]:
def back_propagate(params,input_size,hidden_size,num_labels,X,y,lamda):
    
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    #随机初始化
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size,(input_size +1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels,(hidden_size +1))))
        
    #调用前向传播
    a1, z2, a2, z3, h = forward_propagate(X,theta1,theta2)    
    
    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    
    #计算代价函数
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1-y[i,:]),np.log(1-h[i,:]+ 1e-5))
        J += np.sum(first_term - second_term)
        
    J = J / m
        
    for t in range(m):
        
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        ht = h[t,:]
        yt = y[t,:]
        
        
        #实现反向传播
        d3t = ht - yt
        z2t = np.insert(z2t,0,values = np.ones(1))
        d2t = np.multiply((theta2.T * d3t.T).T,sigmoid_gradient(z2t))
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
    
    #加入正则化
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:]*lamda)/m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:]*lamda)/m  
    
    grad = np.concatenate((np.ravel(delta1),np.ravel(delta2)))
  
    return J,grad

In [14]:
J,grad = back_propagate(params,input_size,hidden_size,num_labels,X,y_onehot,lamda)
print(m)
J,grad.shape

5000


(7.646619469384145, (10285,))

In [19]:
from scipy.optimize import minimize

# fmin = minimize(fun = back_propagate, x0 = params, args = (input_size, hidden_size, num_labels, X, y_onehot, lamda),
#                method = 'TNC', jac = True, options = {'maxiter':250})

fmin = minimize(fun=back_propagate, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, lamda), 
                method='TNC', jac=True, options={'maxiter': 250})

fmin

     fun: 0.2827359324575152
     jac: array([-4.86019045e+00, -2.37847444e-05,  1.94348860e-06, ...,
        2.23294367e+00,  5.42395931e+00, -7.09030828e-02])
 message: 'Max. number of function evaluations reached'
    nfev: 251
     nit: 14
  status: 3
 success: False
       x: array([ 1.36668925, -0.11892372,  0.00971744, ...,  3.68906256,
        1.69051627, -6.21793987])

In [20]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size,(input_size +1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels,(hidden_size +1))))

a1, z2, a2, z3, h = forward_propagate(X,theta1,theta2)    
   
y_pred = np.array(np.argmax(h,axis = 1) + 1)

correct = [1 if a==b else 0 for (a, b) in zip(y_pred,y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))

print('accuracy = {0}%'.format(accuracy * 100))


accuracy = 97.3%
